### 1. create charts with different Color; creating a new excel file "new.xlsx"

In [26]:
from openpyxl import load_workbook
from openpyxl.chart import BarChart, Series, Reference
from openpyxl.chart.label import DataLabelList
wb = load_workbook(filename="Survey.xlsx") 
ws = wb.active



charts_lst=[]
num=1
for ws in wb.worksheets:  
    if len(ws._charts)==1:
        charts_lst.append(num)
    for row in ws.iter_rows(min_col=1, max_col=1):
        if row[0].value is None or row[0].value == "Reason":
            maxrowchart = (row[0].row - 1)
            break
    values = Reference(ws, min_col= 2, min_row=3, max_col = 2, max_row = maxrowchart)
    cats = Reference(ws, min_col=1, max_col=1, min_row=4, max_row= maxrowchart)

# Create object of BarChart class
    chart = BarChart()
    chart.height = 10
    chart.width = 15
    chart.add_data(values, titles_from_data=True)
    chart.set_categories(cats)
                # set the title of the chart
    chart.title = ws["A2"].value

                # the top-left corner of the chart
                # is anchored to cell F2 .
    chart.varyColors = "000F0FFF"
        
    for i in range(1,50):
        s=ws[f"A{i}"].value
        if s == 'Answer Choices': 
            ws.add_chart(chart,"H2")
    num+=1
wb.save("new.xlsx")

In [27]:
print(charts_lst) #contains the question number who has chart

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 79, 80, 81, 82, 84, 85, 86, 87, 88, 92, 93, 94, 95]


In [28]:
len(charts_lst)

87

### Now, each multiple questionsheet has two charts

In [29]:
len(ws._charts)

2

### 2. Convert/Parse EXCEL to WORD "new.docx"
#### Note: this Word doc is not final output: it just parse all the contents from excel

(for now didn't work well by running it myself; instead i use the website below to get the word "new.docx")

In [30]:
#      https://products.aspose.com/cells/python-java/conversion/excel-to-word/

# Convert/Parse EXCEL to WORD

# import jpype     
# import asposecells     
# jpype.startJVM() 
# from asposecells.api import Workbook
# workbook = Workbook("new.xlsx")
# workbook.Save("new.docx")
# jpype.shutdownJVM()
	

### 3. get all the images in "new.docx" and save it to an already-exist folder "test"

In [31]:
# create empty folder "test"
import os
import shutil
dir = 'test'
if os.path.exists(dir):
    shutil.rmtree(dir)
os.makedirs(dir)

### EXTRACT ALL THE IMAGES IN "new.DOC
import docx2txt
#extract text 
text = docx2txt.process(r"new.docx")
#extract text and write images in Temporary Image directory
text = docx2txt.process(r"new.docx",r"test")



### note: in the folder we can see both one-color and multi-color chart for each question;
###       current idea is to only use "even number" image (e.g image 2, image 4, ...)

## NEXT: -figure out step 2 -figure out to put the image into final docx in correct order...   
##      idea: Dictionary, list to store the name of spreadsheets/questions who has multiple choices

In [32]:
import os
lst=[]
for i in os.listdir('test'):
    if (int(i.split('.')[0][5:])%2)==1:
        file=os.path.join('test',i)
        # print(file)
        lst.append(i)
        if os.path.exists(file):
            os.remove(file)
    # else:
    #     os.rename(i.split('.')[0][5:]+'.png','/Users/sara/CPxTCG/excel2word/test copy')
len(lst)  

folder = "test"

for count, filename in enumerate(os.listdir(folder)):
    dst = f"SurveyQuestion {charts_lst[count]}.jpg"
    src =f"{folder}/{filename}"  # foldername/filename, if .py file is outside folder
    dst =f"{folder}/{dst}"
         
        # rename() function will
        # rename all the files
    os.rename(src, dst)


#for i in os.listdir('test'):
    


## further: need to sort the files; then save the file name w/ corresponding question number(charts_lst)

In [33]:
from openpyxl import load_workbook
import os

workbook = load_workbook(filename="Survey.xlsx") 
Sheet_Names = [workbook.sheetnames]
#workbook.sheetnames
#workbook.worksheets
import docx

In [34]:

free_responses={}
for sheet in workbook.worksheets:
    lis=[]
    freechart=[]
    free_responses[sheet.title] = lis
    for i in range(1,50):
        s=sheet[f"A{i}"].value
        res=[]
        if s =='Respondent ID': # if the question is free-response
            

            #for cell in sheet[f'C{i+1}:C1048576']:
            for value in sheet.iter_rows(min_row=i+1,min_col=3, max_col=3,values_only=True):
                if (value[0]!=None):
                    res.append(value[0]) # add res to that single question 
            #print(res)
            free_responses[sheet.title]=res  #add list of res to one question to the big list
            break
            
        
        
            

#free_responses['Question 25']

In [35]:
import docx
doc = docx.Document()

for key,values in free_responses.items():
    length_key = len(free_responses[key])
    ##print(length_key)
    doc.add_paragraph(key)
    ##if question is free response only OR multiple choice free response questions 
    if length_key > 1: 
       
        ##if question has both free response questions AND a chart then add the chart
        if os.path.exists(f'test/Survey{key}.jpg'):
            doc.add_picture(f'test/Survey{key}.jpg')
        ##adds free response answers in bullet format 
        for res in values:
            doc.add_paragraph(res,style='List Bullet 2')
    ##if question does not contain any free response responses 
    if length_key < 1:
        doc.add_picture(f'test/Survey{key}.jpg')
doc.save('output.docx')